# Análise exploratória

&ensp;A análise exploratória dos dados (EDA) investiga conjuntos de dados e pretende resumir suas principais características, empregando métodos de visualização de dados. Além disso, a EDA também proporciona uma melhor compreensão das variáveis do conjunto de dados e das relações entre elas. A etapa de exploração precede o modelo preditivo, pois permite identificar erros óbvios, entender melhor os padrões nos dados, detectar valores discrepantes ou eventos anômalos e encontrar relações interessantes entre as variáveis.

Para mais informações de como rodar, veja o arquivo `orientacao.md`.

* Importação das bibliotecas:

In [25]:
import pandas as pd # pandas
import numpy as np # numpy
import matplotlib.pyplot as plt  # matplotlib
import seaborn as sns  # seaborn
from sklearn.ensemble import IsolationForest

* Leitura e visualização do DataFrame:

In [ ]:
df = pd.read_excel('./data.xlsx') # leitura do arquivo
df.head() # visualização do Data Frame

## Estatísticas descritivas básicas

In [ ]:
numeric_cols = df[["Valor Pago Sinistro", "Quantidade", "No Ano Mes"]] # seleção de colunas numéricas
basic_stats = numeric_cols.describe()
basic_stats

Outras estatísticas descritivas:
* Variância
* Desvio padrão
* Coeficiente de Variação
* Assimetria
* Curtose
* Correlação

In [28]:
variances = numeric_cols.var()
standart_deviation = numeric_cols.std()
coef_variation = standart_deviation / numeric_cols.mean()
skew = numeric_cols.skew()
kurtosis = numeric_cols.kurtosis()
correlation = numeric_cols.corr()

## Pré-processamento de dados

&ensp;O pré-processamento de dados envolve a preparação dos dados brutos para que possam ser usados de forma eficaz por algoritmos de modelagem. Essa etapa garante que os dados estejam em um formato limpo, consistente e pronto para análise ou construção de modelos.

&ensp;Em primeira análise, foi descoberto a presença de dois valores com a mesma atribuição qualitativa, "UNIPAR INDUPA DO BRASIL S.A" e "UNIPAR INDUPA DO BRASIL S.A.". 

In [ ]:
df["Nome Empresa Sinistro"].value_counts()

In [ ]:
df["Nome Empresa Sinistro"] = df["Nome Empresa Sinistro"].replace("UNIPAR INDUPA DO BRASIL S.A.", "UNIPAR INDUPA DO BRASIL S.A")
df["Nome Empresa Sinistro"].value_counts()

&ensp;Além disso, foi verificada a presença de variáveis com valor nulo (Missing Values), onde o resultado foi positivo:

In [ ]:
df.isnull().sum()

&ensp;A coluna "No Ano Mês" foi normalizada para adequar strings de data para objetos datetime:

In [ ]:
df["no-ano-mes_stardardized"] = pd.to_datetime(df["No Ano Mes"])
df.head()

&ensp;A coluna "Faixa-Etária Nova Sinistro", classificada como categórica ordenada, foi codificada por meio do método Label Encoding:

In [ ]:
df["faixa-etaria_encoded"] = df["Faixa-Etária Nova Sinistro"].astype('category').cat.codes
df["faixa-etaria_encoded"].value_counts()
df.head()

&ensp;As variáveis "Valor Pago Sinistro" e "Quantidade" foram padronizadas usando o método _Standart Scaler_, utilizado para padronizar as features de um conjunto de dados, ou seja, para transformá-las de modo que tenham média igual a 0 e desvio padrão igual a 1.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df.head()

## Identificação de outliers

&ensp;A identificação de outliers é uma etapa fundamental, pois esses pontos podem distorcer análises estatísticas e prejudicar o desempenho do modelo. </br>
&ensp;Primeiramente, foram identificadas, na base de dados as variáveis numéricas que apresentam um padrão ou tendencia que se relacionar com as outras linhas (variáveis que poderiam ter outliers), que foram as colunas de 'Valor Pago Sinistro' e 'Quantidade', elas se referem ao valor de pagamento efetuado e a quantidade de serviços que aquele pagamento representa, respectivamente.</br>
&ensp;Após isso, foi feita uma análise gráfica de cada uma delas para identificar possíveis dados que se destacam ou fogem do padrão dos outros.

### Coluna 'Valor Pago Sinistro'

&ensp; Para a visualização gráfica da coluna 'Valor Pago Sinistro, é possível fazer um gráfico de caixas (boxplot), onde os pontos muito distoantes são outliers.


In [ ]:
sns.boxplot(x=df['Valor Pago Sinistro'])
plt.show()


&ensp;Após analisar o gráfico, é possivel perceber que no intervalo de 0 até um pouco antes de 100 mil os pontos quase formam uma linha contínua, algo que para de se cumprir quando o valor pago de sinistro ultrapassa os 100 mil reais. Logo, pode-se analisar que essas ocorrências são muito raras e distôam da maioria. Ainda assim, não podem ser considerados _outliers_, pois isso depende da forma com a qual nós os utilizariamos. Em certas situações, como ao calcular a média dos valores de sinsitros, eles podem ser considerados, mas só por aparecerem em situações raras, não podemos negligênciar sua ocorrência e importância na base de dados.<br/>
&ensp;Dito isso, segue a comprovação da quantidade dessas ocorrências acima de 100 mil reais:

In [ ]:
ocorrencia = df['Valor Pago Sinistro'] > 100000
contagem_de_ocorrencia = ocorrencia.value_counts()
print(contagem_de_ocorrencia)

&ensp;Logo, apenas 16 linhas de cem mil, possuem valores acima dos 100 mil reais de valor pago em sinistro. Agora, vamos mostrar como eles afetam a média, e depois qual o seu impacto no valor total gasto em sinistro, para comprovar nossa tese em relação a esses "_outliers_".

In [ ]:
media_antes = df['Valor Pago Sinistro'].mean()
print("A média sem eles é: " + str(media_antes))

media_depois = df[df['Valor Pago Sinistro'] < 100000]['Valor Pago Sinistro'].mean()
print('A média depois é: ' + str(media_depois))

&ensp;Agora, o impacto no valor total:

In [ ]:
valor_total = df['Valor Pago Sinistro'].sum()
soma_ocorrencia = df[df['Valor Pago Sinistro'] > 100000]['Valor Pago Sinistro'].sum()
impacto = (soma_ocorrencia / valor_total) * 100

# Impressão dos valores para verificação e do impacto formatado
print(f'Valor Total: {valor_total}')
print(f'Soma das Ocorrências (valores > 100000): {soma_ocorrencia}')
print(f'É um impacto de {impacto:.2f}%')

Sendo assim, essa alteração pode desempenhar um papel significativo quando formos treinar o nosso modelo preditivo.

### Coluna 'Quantidade'

&ensp; Para visualizar melhor os outliers da coluna 'Quantidade', fizemos outro boxplot. Nele, é possível identificar alguns dados que possuem quantidades iguais a zero.

In [ ]:
sns.boxplot(x=df['Quantidade'])
plt.show()

&ensp; Sendo assim, esses dados não se encaixam no número de quantidades reais e possíveis. Portanto, esses dados devem ser alterados. Pensando nisso, calculamos a moda - o valor que mais se repete - dessa coluna, que foi igual a 1, como pode ser observado na operação feita abaixo.

In [ ]:
moda = df["Quantidade"].mode()
moda.index = ['Moda:']
print(moda)


&ensp; Após calcularmos a moda, substituímos os valores que eram considerados outliers pelo grupo, por 1. Vale ressaltar que essa decisão foi tomada considerando o impacto dos dados. Dessa forma, buscamos minimizar os efeitos desses valores nas análises e no modelo preditivo que estaremos trabalhando na terceira sprint.

In [41]:
df['Quantidade'] = df['Quantidade'].replace([0], 1)

&ensp; Com as alterações feitas, podemos visualizar o seguinte gráfico como resultado, no qual é possível observar as mudanças. Os outliers foram corrigidos para que não impactem negativamente o projeto.


In [ ]:
sns.boxplot(x=df['Quantidade'])
plt.show()


### Inequidade do código do serviço com a descrição

&ensp; A coluna 'Código do Serviço' apresenta uma inequidade com a coluna 'Descrição do Serviço', pois, ao analisar a base de dados, percebemos que a contagem de códigos de serviço é menor que a contagem de descrições de serviço. Isso ocorre porque um mesmo código de serviço foi aplicado por engano a diferentes descrições de serviço. Para corrigir essa inequidade, foi feita uma análise para identificar quais códigos de serviço foram aplicados a diferentes descrições de serviço. A partir dessa análise, foi possível identificar os códigos de serviço que foram aplicados a diferentes descrições de serviço e corrigir essa inequidade.

In [ ]:
# Inicializa o mapeamento com o dicionário correto
mapeamento_codigo = {}

# Popula o dicionário de mapeamento
for index, row in df.iterrows():
    if row["Codigo Servico Sinistro"] not in mapeamento_codigo:
        mapeamento_codigo[row["Codigo Servico Sinistro"]] = row["Descricao Servico Sinistro"]
        mapeamento_codigo[row["Descricao Servico Sinistro"]] = row["Codigo Servico Sinistro"]

# Função para verificar inconsistências
def verificar_inconsistencia(row):
    codigo = row["Codigo Servico Sinistro"]
    descricao = row["Descricao Servico Sinistro"]
    
    # Verifica se o código existe no mapeamento e se a descrição está correta
    return codigo in mapeamento_codigo and descricao != mapeamento_codigo[codigo]

inconsistencias = df[df.apply(verificar_inconsistencia, axis=1)]
# Arrumando inconsistências
for index, row in inconsistencias.iterrows():
    df.at[index, "Codigo Servico Sinistro"] = mapeamento_codigo[row["Descricao Servico Sinistro"]]

# Verificando se ainda existem inconsistências
inconsistencias = df[df.apply(verificar_inconsistencia, axis=1)]
inconsistencias

### Utilizando novos dados de IA generativa
&emsp;Nesse processo, utilizamos uma técnica de IA generativa para gerar novos dados que possam ser utilizados para melhorar a performance do modelo preditivo. Os dados enviados foram a descrição do serviço, onde foi pedido que ela alocasse em categorias menos específicas e se o exame era relacionado uma doença em potencial ou não. A partir disso, foi gerado um novo dataset com essas informações, que é utilizado com novas features para o modelo preditivo. Para mais detalhes, consulte a documentação.

In [44]:
import pandas as pd

# Carrega o JSON de categorias
categorias = pd.read_json('./assets/categorias.json')

# Itera sobre cada categoria (coluna) do DataFrame de categorias
for categoria in categorias.columns:
    # Atualiza as colunas "Doença relacionada" e "Tipo de Serviço" com valores da categoria
    df.loc[df["Descricao Servico Sinistro"] == categoria, "Doença relacionada"] = categorias[categoria].iloc[1]
    df.loc[df["Descricao Servico Sinistro"] == categoria, "Tipo de Serviço"] = categorias[categoria].iloc[0]

Além disso, outro passo importante foi trocar os textos das colunas adicionadas para números, para que o modelo possa interpretar esses dados. Para isso, foi utilizado o método Label Encoding.

In [ ]:
# Convertendo para label encoded para o modelo de machine learning
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
df[['doenca_relacionadas_encoded', 'tipo_servico_encoded']] = df[['Doença relacionada', 'Tipo de Serviço']].apply(labelEncoder.fit_transform)
df.head()

Padronizando a data para o formato datetime e depois para inteiro, para que o modelo possa interpretar esses dados.

In [46]:
df["Dt Data Sinistro"] = pd.to_datetime(df["Dt Data Sinistro"])
df["Dt Data Sinistro Padronizada"] = df["Dt Data Sinistro"].astype('int64')

Coletando o dia da semana em que o sinistro foi realizado, assim, podemos verificar a viabilidade de se criar um modelo que leva em consideração o dia da semana em que o sinistro foi realizado.

In [47]:
df["Dia da Semana Sinistro"] = df["Dt Data Sinistro"].dt.day_name("portuguese")
df["dia_da_semana_sinistro_encoded"] = df["Dia da Semana Sinistro"].astype('category').cat.codes

Também foi verificado o impacto de feriados próximos ao dia do sinistro, para verificar se há uma relação entre a ocorrência de sinistros e feriados.

In [48]:
feriados_2023 = pd.read_json('./assets/feriados_2023.json')
feriados_2024 = pd.read_json('./assets/feriados_2024.json')

# Concatena os feriados de 2023 e 2024
feriados = pd.concat([feriados_2023, feriados_2024])

feriados["date"] = pd.to_datetime(feriados["date"])


# Verifica se a data do sinistro é um feriado em um range de 3 dias para mais ou para menos
def verificar_feriado(row):
    data = row["Dt Data Sinistro"]
    return feriados["date"].between(data - pd.Timedelta(days=3), data + pd.Timedelta(days=3)).any()
df["Feriado Próximo"] = df.apply(verificar_feriado, axis=1)

Deixando a coluna 'Elegibilidade' em formato numérico, para que o modelo possa interpretar esses dados.

In [49]:
df["elegibilidade_sinistro_encoded"] = df["Elegibilidade Sinistro"].astype('category').cat.codes

Deixando a coluna 'Sexo Sinistro' para numérica para que o modelo possa interpretar esses dados.

In [50]:
df["sexo_encoded"] = df["Sexo Sinistro"].astype('category').cat.codes

A coluna 'Descricao Plano Sinistro' foi transformada em numérica para que o modelo possa interpretar esses dados.

In [51]:
df["descricao_plano_sinistro_encoded"] = df["Descricao Plano Sinistro"].astype('category').cat.codes

Aqui foi feita a transformação da variável 'Descricao do Serviço' para numérica, apesar dela possuir uma coluna que transforme ela em númerica, os valores são distoantes e não são úteis para o modelo, então foi feita uma nova transformação.

In [52]:
df["descricao_servico_sinistro_encoded"] = df["Descricao Servico Sinistro"].astype('category').cat.codes


Padronizando as variáveis que tem valores muito altos, para que o modelo possa interpretar esses dados.

In [53]:
df[['valor-pago-sinistro_standardized', 'quantidade_standardized', 'descricao_servico_sinistro_encoded_standardized', 'doenca_relacionadas_encoded_standardized', 'tipo_servico_encoded_standardized', 'dia_da_semana_sinistro_encoded_standardized', 'faixa-etaria_encoded_standardized', 'data_sinistro_standardized']] = scaler.fit_transform(df[["Valor Pago Sinistro", "Quantidade", "descricao_servico_sinistro_encoded", "doenca_relacionadas_encoded", "tipo_servico_encoded", "dia_da_semana_sinistro_encoded", "faixa-etaria_encoded", "Dt Data Sinistro Padronizada"]])

In [ ]:
df.head()

In [ ]:
df.describe()

### Exportando o dataset

In [56]:
df.to_csv('./data_updated.csv', index=False)

### Exportando os objetos de padronização

In [57]:
import pickle

# Salva o scaler em um arquivo
with open('./assets/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Salva o label encoder em um arquivo
with open('./assets/label_encoder.pkl', 'wb') as file:
    pickle.dump(labelEncoder, file)